In [33]:
from procesador_guia_ocr import ProcesadorGuiaOCR
from conexion_sql import get_connection
from conexion_sql import get_sqlalchemy_engine
import pandas as pd
from datetime import datetime

# Validar  disponibilidad de ruta \\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\05\12\2\

class main:
    def __init__(self):
        hora_inical = datetime.now()
        
    def EjecucionPaso01_Devoluciones(): 
        conexion = get_connection()
        if conexion:
            try:
                consulta_script = f'''
                                    SELECT ---------------top 10
                                        NUMERO_GUIA,
                                        PATH_IMAGEN
                                        FROM  [TMP].[TEMU_GUIAS_DEVOLUCIONES_01] WITH(NOLOCK) 
                                        WHERE NOMBRE_ZONA_URBA <> 'D RF VOLANTES SUR 20'
                                        AND NUMERO_GUIA NOT IN (SELECT numero_guia from [TMP].[TEMU_GUIAS_DEVOLUCIONES_02] WITH(NOLOCK))
                                        AND PATH_IMAGEN IS NOT NULL
                                        OPTION(FAST 1)                                     
                                        '''
                df_SP02 = pd.read_sql(consulta_script, conexion)
                           
                print("Procedimiento ejecutado correctamente.")
                print(datetime.now())
            except Exception as e:
                print("Error al ejecutar la consulta:", e)
            finally:
                print("Conexión cerrada correctamente.")
            return(df_SP02)    

if __name__ == "__main__":
    RUTA_TESSERACT = r'\\t_serv-dbi01\T\App_Costos_Darwin\Analizador_IMG\tesseract.exe'
    
    errores = 0 
    guias = main.EjecucionPaso01_Devoluciones()    
    
    numero_registros = len(guias)  

    i = 1 
    guias_df = pd.DataFrame(columns=[
    "ruta_imagen",
    "numero_guia",
    "codigo_barras",
    "tiene_devolucion",
    "estado_devolucion",
    "tiene_codigo_barras"
        ])
    
    for indice, fila in guias.iterrows():
        GUIA_PATH = fila['PATH_IMAGEN']
        # print(GUIA_PATH)        
        #GUIA_PATH = r'\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\06\18\2\045\2247178045.TIF'

        try:
            procesador = ProcesadorGuiaOCR(RUTA_TESSERACT)
            df,errores = procesador.procesar_imagen(GUIA_PATH)
            df['numero_guia'] = fila['NUMERO_GUIA']
        except:
            errores = errores + 1
        finally:
            if len(df)>=1:               
                guias_df = pd.concat([guias_df, df], ignore_index=True)        
            i = i+1
        print(f"\rRegistros procesados: {i} / {numero_registros}   \U0000274C CON PRESENCIA DE ERRORES  {errores}", end="", flush=True)


    # print(len(guias_df))
    print('\n\n')
    print(f'total registros generados con error: {numero_registros}')
    print('\n\n')
    
    # Truncar tabla de [TMP].[TEMU_GUIAS_DEVOLUCIONES_02]
    conexion = get_connection()
    if conexion:
        try:
            cursor = conexion.cursor()
            consulta_SP = '''
                                EXEC [TMP].[TEMU_SP_GUIAS_DEVOLUCIONES_02]
                        '''
            cursor.execute(consulta_SP) 
            # Asegurar que Python consuma todos los result sets y espere hasta el final
            ct_rs = 0
            while cursor.nextset():
                ct_rs = 1+ct_rs
                print("Ejecucion numero de result sets: "+str(ct_rs))    
                pass
            print("Script ejecutado correctamente.")
            print(datetime.now())
            conexion.commit()    
        except Exception as e:
            print("Error al ejecutar la consulta:", e)
        finally:
            cursor.close()
            conexion.close()
            print("Conexión cerrada correctamente.")

    print(f'Numero de errores {errores}')

    ##guias_df = guias_df[pd.to_numeric(guias_df['numero_guia'], errors='coerce').notna()]

    #INSERTAR DATOS EN TABLA DE PASO CON CALCULO DE ESTADISTICOS
    batch_size = 5000
    conexion = get_connection()
    if conexion:
        try:
            conexion.autocommit = True  # Mejora la velocidad eliminando commits explícitos
            cursor = conexion.cursor()
            cursor.fast_executemany = True  # Activa inserción rápida en lotes
            
            data = [tuple(row) for row in guias_df.itertuples(index=False, name=None)]
            total_rows = len(data)

            sql = '''INSERT INTO [TMP].[TEMU_GUIAS_DEVOLUCIONES_02] WITH (TABLOCK)
                    VALUES (?,	?,	?,	?,	?,	?)'''

            for i in range(0, total_rows, batch_size):
                batch = data[i:min(i + batch_size, total_rows)]
                cursor.executemany(sql, batch)  # Se ejecuta sin necesidad de commit explícito
                
                porcentaje = int((i + len(batch)) / total_rows * 100)
                print(f"\r{porcentaje}% Insertados {i + len(batch)} de {total_rows} registros...", end="", flush=True)

            print("\nInserción completada correctamente.")  
            print(datetime.now())
        except Exception as e:
            print("\nError al ejecutar la consulta:", e)
        finally:
            # cursor.close()
            # conexion.close()
            print("Conexión cerrada correctamente.")    





Conexión establecida con SQL Server


C:\Users\john.cruz\AppData\Local\Temp\ipykernel_10916\1044115789.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SP02 = pd.read_sql(consulta_script, conexion)


Procedimiento ejecutado correctamente.
2025-12-23 08:33:10.958575
Conexión cerrada correctamente.



total registros generados con error: 0



Conexión establecida con SQL Server
Script ejecutado correctamente.
2025-12-23 08:33:11.122089
Conexión cerrada correctamente.
Numero de errores 0
Conexión establecida con SQL Server

Inserción completada correctamente.
2025-12-23 08:33:11.142411
Conexión cerrada correctamente.


In [22]:
##guias_df = guias_df[pd.to_numeric(guias_df['numero_guia'], errors='coerce').isna()]
guias_df = guias_df[pd.to_numeric(guias_df['numero_guia'], errors='coerce').notna()]


In [30]:
display(guias_df)

,ruta_imagen,numero_guia,codigo_barras,tiene_devolucion,estado_devolucion,tiene_codigo_barras
0,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\07\2\732\2246650732.TIF,2.246651e+09,None,True,SÍ,False
1,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\01\30\2\737\2246651737.TIF,2.246652e+09,2\46650337,True,SÍ,True
2,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\25\2\782\2246651782.TIF,2.246652e+09,2246651782,True,SÍ,True
3,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\10\2\071\2246652071.TIF,2.246652e+09,2246652071,True,SÍ,True
4,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\04\2\058\2246653058.TIF,2.246653e+09,2246653058,True,SÍ,True
5,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\04\2\622\2246653622.TIF,2.246654e+09,2246653622,False,NO,True
6,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\06\2\837\2246653837.TIF,2.246654e+09,2246653837,True,SÍ,True
7,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\12\2\961\2246653961.TIF,2.246654e+09,None,False,NO,False
8,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\25\2\661\2246655661.TIF,2.246656e+09,2246655661,True,SÍ,True
9,\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\05\2\259\2246656259.TIF,2.246656e+09,None,True,SÍ,False


In [29]:
import time
import sys

# Ejemplo con una lista de registros
registros = ["guia1", "guia2", "guia3", "guia4", "guia5"] # Tu lista real aquí
total = len(registros)

print("🚀 Iniciando conteo de registros...")

for i, registro in enumerate(registros, 1):
    # Procesar aquí tu registro (DB, OCR, etc.)
    time.sleep(5)
    # Imprime el contador: \r vuelve al inicio, end="" evita el salto de línea
    print(f"\rRegistros procesados: {i} / {total}", end="", flush=True)

print("\n✅ Fin del proceso.")

🚀 Iniciando conteo de registros...
Registros procesados: 5 / 5
✅ Fin del proceso.


In [24]:
# Truncar tabla de [TMP].[TEMU_GUIAS_DEVOLUCIONES_02]
conexion = get_connection()
if conexion:
    try:
        cursor = conexion.cursor()
        consulta_SP = '''
                            ---TRUNCATE TABLE [TMP].[TEMU_GUIAS_DEVOLUCIONES_02]
                    '''
        cursor.execute(consulta_SP) 
        # Asegurar que Python consuma todos los result sets y espere hasta el final
        ct_rs = 0
        while cursor.nextset():
            ct_rs = 1+ct_rs
            print("Ejecucion numero de result sets: "+str(ct_rs))    
            pass
        print("Script ejecutado correctamente.")
        print(datetime.now())
        conexion.commit()    
    except Exception as e:
        print("Error al ejecutar la consulta:", e)
    finally:
        cursor.close()
        conexion.close()
        print("Conexión cerrada correctamente.")


#INSERTAR DATOS EN TABLA DE PASO CON CALCULO DE ESTADISTICOS
batch_size = 5000
conexion = get_connection()
if conexion:
    try:
        conexion.autocommit = True  # Mejora la velocidad eliminando commits explícitos
        cursor = conexion.cursor()
        cursor.fast_executemany = True  # Activa inserción rápida en lotes
        
        data = [tuple(row) for row in guias_df.itertuples(index=False, name=None)]
        total_rows = len(data)

        sql = '''INSERT INTO [TMP].[TEMU_GUIAS_DEVOLUCIONES_02] WITH (TABLOCK)
                VALUES (?,	?,	?,	?,	?,	?)'''

        for i in range(0, total_rows, batch_size):
            batch = data[i:min(i + batch_size, total_rows)]
            cursor.executemany(sql, batch)  # Se ejecuta sin necesidad de commit explícito
            
            porcentaje = int((i + len(batch)) / total_rows * 100)
            print(f"\r{porcentaje}% Insertados {i + len(batch)} de {total_rows} registros...", end="", flush=True)

        print("\nInserción completada correctamente.")  
        print(datetime.now())
    except Exception as e:
        print("\nError al ejecutar la consulta:", e)
    finally:
        # cursor.close()
        # conexion.close()
        print("Conexión cerrada correctamente.")    


Conexión establecida con SQL Server
Script ejecutado correctamente.
2025-12-23 08:11:07.915314
Conexión cerrada correctamente.
Conexión establecida con SQL Server
100% Insertados 1447 de 1447 registros...
Inserción completada correctamente.
2025-12-23 08:11:08.494659
Conexión cerrada correctamente.


In [17]:
from procesador_guia_ocr import ProcesadorGuiaOCR

RUTA_TESSERACT = r'\\t_serv-dbi01\T\App_Costos_Darwin\Analizador_IMG\tesseract.exe'
GUIA_PATH = r'\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\06\18\2\045\2247178045.TIF'

procesador = ProcesadorGuiaOCR(RUTA_TESSERACT)

df = procesador.procesar_imagen(GUIA_PATH)

print("\n DATAFRAME RESULTANTE\n")
display(df)


❌ Error procesando imagen: No se pudo cargar la imagen

 DATAFRAME RESULTANTE



,ruta_imagen,numero_guia,codigo_barras,tiene_devolucion,estado_devolucion,tiene_codigo_barras


In [ ]:
from procesador_guia_ocr import ProcesadorGuiaOCR

RUTA_TESSERACT = r'\\t_serv-dbi01\T\App_Costos_Darwin\Analizador_IMG\tesseract.exe'
GUIA_PATH = r'\\servient.com.co\imgmasnal\IMGMASNAL\NACIONAL\02-05-2021\2025\02\10\2\655\2246650655.TIF'

procesador = ProcesadorGuiaOCR(RUTA_TESSERACT)

df = procesador.procesar_imagen(GUIA_PATH)

print("\n DATAFRAME RESULTANTE\n")
display(df)


